In [70]:
STATE_NAMES = ["Alaska", "Alabama", "Arkansas", "Arizona", "California",
"Colorado", "Connecticut", "Delaware", "Florida", "Georgia",
"Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana",
"Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi",
"Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey",
"New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
"Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia",
"Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

STATE_NAMES_WITH_UNITED_STATES = STATE_NAMES[:]
STATE_NAMES_WITH_UNITED_STATES.append("United States")


In [73]:
import pandas as pd

df = pd.read_csv("2017_us_state_finances.csv")
df['Description'] = df['Description'].str.strip()

df.rename(columns = {'United States Total':'United States'}, inplace = True)

# Retains only columns required
df = df[["Description"] + STATE_NAMES_WITH_UNITED_STATES]
# df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Drops rows relating to revenue sources
df.drop(df.index[0:66], inplace=True)

social = ["Public welfare", "Hospitals", "Health", "Employment security administration", "Veterans' services"]
educ = ["Education", "Libraries"]
govt = ["Financial administration", "Judicial and legal", "General public buildings", "Other governmental administration"]
transport = ["Highways", "Air transportation (airports)", "Parking facilities", "Sea and inland port facilities"]
others = ["Utility expenditure", "Expenditure1"]

# Retains only required rows
df = df[df["Description"].isin(social + educ + govt + transport + others)]

# # Education (Education + Libraries) - 61
# # Health & Social Services (Public welfare + Hospitals + Health + Security + Employment security administration +  Veterans' services) - 62
# # Government Administration (Financial administration + Judicial and legal + General public buildings + Other governmental administration) - 92
# # Utilities (Utility expenditure) - 22
# # Transportation (Highways, Air transportation (airports), Parking facilities, Sea and inland port facilities) - 48/49

# # Transposes dataframe and rename columns
df = df.transpose()
df.columns = df.iloc[0]
df = df[1:]
df.reset_index(inplace=True)
df.rename(columns = {'Expenditure1':'State Expenditure', 
"index" : "State", "Utility expenditure" : "Utilities"}, inplace = True)

for col in [col for col in df.columns]:
    if col != "State":
        df[col] = df[col].str.replace(',','')
        df[col] = df[col].astype(int)

df["Health and Social Services"] = df[social].sum(axis=1)
df["Education Related"] = df[educ].sum(axis=1)
df["Public Administration"] = df[educ].sum(axis=1)
df["Transportation"] = df[transport].sum(axis=1)
df.drop(columns = social + educ + govt + transport, inplace=True)

required_col_names = [col for col in df.columns[1:]]

for col in [col for col in required_col_names]:
    if col == "State Expenditure":
        col_name = col + " as % of US Expenditure"
        df["State Total as % of US Total"] = \
            df.apply(lambda x : (x[col] / int(df.loc[df["State"] ==
                "United States", col])) * 100, axis = 1)

    else:
        col_name = col + " (State Expenditure as % of Total US)"
        df[col + " (State as % of US)"] = \
            df.apply(lambda x : (x[col] / int(df.loc[df["State"] ==
                "United States", col])) * 100, axis = 1)

        df[col + " (% of Total Expenditure)"] = (df[col] /
        df["State Expenditure"] * 100)

df.set_index(["State"], inplace = True)

df

Description,State Expenditure,Utilities,Health and Social Services,Education Related,Public Administration,Transportation,State Total as % of US Total,Utilities (State as % of US),Utilities (% of Total Expenditure),Health and Social Services (State as % of US),Health and Social Services (% of Total Expenditure),Education Related (State as % of US),Education Related (% of Total Expenditure),Public Administration (State as % of US),Public Administration (% of Total Expenditure),Transportation (State as % of US),Transportation (% of Total Expenditure)
State,,,,,,,,,,,,,,,,,
Alaska,14939769,799060,3298694,3316918,3316918,1950228,0.407094,0.343766,5.348543,0.337343,22.079953,0.323119,22.201936,0.323119,22.201936,0.904204,13.053937
Alabama,46961537,3174122,14205075,14102709,14102709,2689922,1.279656,1.365548,6.758982,1.452690,30.248318,1.373823,30.030340,1.373823,30.030340,1.247155,5.727926
Arkansas,27688857,987682,9077837,8413108,8413108,2151829,0.754494,0.424913,3.567074,0.928350,32.785163,0.819567,30.384454,0.819567,30.384454,0.997674,7.771462
Arizona,60084606,5481940,16158938,16789089,16789089,3099014,1.637246,2.358401,9.123701,1.652503,26.893641,1.635518,27.942413,1.635518,27.942413,1.436827,5.157750
California,569430129,43593282,165956421,135421829,135421829,23091879,15.516410,18.754386,7.655598,16.971623,29.144299,13.192187,23.781992,13.192187,23.781992,10.706319,4.055261
Colorado,60919223,4513782,12181924,17726200,17726200,4139615,1.659989,1.941887,7.409454,1.245791,19.996847,1.726807,29.097876,1.726807,29.097876,1.919291,6.795252
Connecticut,41048798,1423979,6380256,13647313,13647313,2317356,1.118539,0.612614,3.468991,0.652480,15.543101,1.329460,33.246559,1.329460,33.246559,1.074419,5.645369
Delaware,12100844,540908,3124465,4075944,4075944,915689,0.329736,0.232706,4.470002,0.319525,25.820224,0.397060,33.683138,0.397060,33.683138,0.424550,7.567150
Florida,177089394,10988094,43797757,43350737,43350737,14737625,4.825511,4.727218,6.204829,4.479001,24.732005,4.223034,24.479578,4.223034,24.479578,6.832952,8.322139


In [74]:
pop_df = pd.read_csv("us_census_population.csv")
pop_df = pop_df.iloc[:,0:2]
pop_df.columns = ["State", "2020 Census Population"]
pop_df['State'] = pop_df['State'].str.strip()
pop_df = pop_df[pop_df["State"].isin(STATE_NAMES_WITH_UNITED_STATES)]

pop_df["2020 Census Population"] = pop_df["2020 Census Population"].str.replace(',','')
pop_df["2020 Census Population"] = pop_df["2020 Census Population"].astype(int)

pop_df.set_index("State", inplace=True)

pop_df


,2020 Census Population
State,
United States,331449520
Alabama,5024356
Alaska,733378
Arizona,7151507
Arkansas,3011555
California,39538245
Colorado,5773733
Connecticut,3605942
Delaware,989957


In [75]:
poverty_df = pd.read_csv("us_poverty_by_state.csv")
poverty_df = poverty_df.iloc[:,0:2]
poverty_df.columns = ["State", "3-Year Average Poverty Rate (2018-2020)"]
poverty_df['State'] = poverty_df['State'].str.strip()
poverty_df = poverty_df[poverty_df["State"].isin(STATE_NAMES_WITH_UNITED_STATES)]

poverty_df["3-Year Average Poverty Rate (2018-2020)"] = poverty_df["3-Year Average Poverty Rate (2018-2020)"].astype(float)

poverty_df.set_index("State", inplace=True)

poverty_df

,3-Year Average Poverty Rate (2018-2020)
State,
United States,11.2
Alabama,14.6
Alaska,12.2
Arizona,11.2
Arkansas,14.7
California,11.0
Colorado,9.3
Connecticut,9.9
Delaware,8.1


In [76]:
df_lst = [(poverty_df, []), (pop_df, [])]

def combine_dataframes_by_state(main_df, df_lst):
    """
    Recursively concatenates multiple panda dataframes (with "State" 
    as the index) with only the required columns

    Inputs:
        df_lst (lst of tuples): (df, [cols to extract])
        ### If extracting all columns, [cols to extract] should be 
        an empty list ###
        
    Returns:
        final_df (pandas series): concatenated pandas dataframes
    """
    if len(df_lst) == 0:
        return main_df

    other_df, col_lst = df_lst.pop()
    if col_lst != []:
        new_df = main_df.merge(other_df[col_lst], on="State")
    else:
        new_df = main_df.merge(other_df, on="State")
    
    return combine_dataframes_by_state(new_df, df_lst)

combine_dataframes_by_state(df, df_lst)

,State Expenditure,Utilities,Health and Social Services,Education Related,Public Administration,Transportation,State Total as % of US Total,Utilities (State as % of US),Utilities (% of Total Expenditure),Health and Social Services (State as % of US),Health and Social Services (% of Total Expenditure),Education Related (State as % of US),Education Related (% of Total Expenditure),Public Administration (State as % of US),Public Administration (% of Total Expenditure),Transportation (State as % of US),Transportation (% of Total Expenditure),2020 Census Population,3-Year Average Poverty Rate (2018-2020)
State,,,,,,,,,,,,,,,,,,,
Alaska,14939769,799060,3298694,3316918,3316918,1950228,0.407094,0.343766,5.348543,0.337343,22.079953,0.323119,22.201936,0.323119,22.201936,0.904204,13.053937,733378,12.2
Alabama,46961537,3174122,14205075,14102709,14102709,2689922,1.279656,1.365548,6.758982,1.452690,30.248318,1.373823,30.030340,1.373823,30.030340,1.247155,5.727926,5024356,14.6
Arkansas,27688857,987682,9077837,8413108,8413108,2151829,0.754494,0.424913,3.567074,0.928350,32.785163,0.819567,30.384454,0.819567,30.384454,0.997674,7.771462,3011555,14.7
Arizona,60084606,5481940,16158938,16789089,16789089,3099014,1.637246,2.358401,9.123701,1.652503,26.893641,1.635518,27.942413,1.635518,27.942413,1.436827,5.157750,7151507,11.2
California,569430129,43593282,165956421,135421829,135421829,23091879,15.516410,18.754386,7.655598,16.971623,29.144299,13.192187,23.781992,13.192187,23.781992,10.706319,4.055261,39538245,11.0
Colorado,60919223,4513782,12181924,17726200,17726200,4139615,1.659989,1.941887,7.409454,1.245791,19.996847,1.726807,29.097876,1.726807,29.097876,1.919291,6.795252,5773733,9.3
Connecticut,41048798,1423979,6380256,13647313,13647313,2317356,1.118539,0.612614,3.468991,0.652480,15.543101,1.329460,33.246559,1.329460,33.246559,1.074419,5.645369,3605942,9.9
Delaware,12100844,540908,3124465,4075944,4075944,915689,0.329736,0.232706,4.470002,0.319525,25.820224,0.397060,33.683138,0.397060,33.683138,0.424550,7.567150,989957,8.1
Florida,177089394,10988094,43797757,43350737,43350737,14737625,4.825511,4.727218,6.204829,4.479001,24.732005,4.223034,24.479578,4.223034,24.479578,6.832952,8.322139,21538226,12.8
